# Credit Value at Risk

In [1]:
import numpy as np
import pandas as pd
import requests
import io
from scipy.stats import binom, norm

from math import exp
from math import log
from math import sqrt
import locale

In [2]:
_ = locale.setlocale(locale.LC_ALL, '')

### Exercise 21.6
Answer from the textbook:
* The pobability of 6 or more defaults is 0.0005

In [3]:
num_loans = 100
pr_default = .01
num_defaults = 6
print(f'Probability of {num_defaults:d} or more defaults: {(1 - binom.cdf(num_defaults-1, num_loans, pr_default)):.4f}')

Probability of 6 or more defaults: 0.0005


### Exercise 21.7
Answer from the textbook:
* The pobability of 6 or more defaults is 0.0021

In [4]:
pds = [.005, .015]
pr_default = sum([1 - binom.cdf(num_defaults-1, num_loans, pd) for pd in pds]) / len(pds)
print(f'Probability of {num_defaults:d} or more defaults: {pr_default:.4f}')

Probability of 6 or more defaults: 0.0021


### Exercise 21.8
Answer from the textbook (confirmed by the author to be incorrect):
* The autocorrelation is quite high at 0.546

Retrieving Table 11.6 from the author's website.

In [5]:
with requests.get('http://www-2.rotman.utoronto.ca/~hull/calcs/Chapter11DefaultRates5e.xls') as s:
    df = pd.read_excel(io.BytesIO(s.content), index_col=0, header=5,
                       usecols='A,C', names=['Year', 'Defaul Rate'], parse_dates=True)

In [6]:
df = df.dropna()
df = df.asfreq('AS')

In [7]:
df

,Defaul Rate
Year,
1970-01-01,0.02631
1971-01-01,0.00286
1972-01-01,0.00453
1973-01-01,0.00456
1974-01-01,0.00275
1975-01-01,0.00361
1976-01-01,0.00176
1977-01-01,0.00354
1978-01-01,0.00354


In [8]:
print(f'The autocorrelation is: {df.iloc[:,0].autocorr():.3f}')

The autocorrelation is: 0.563


### Exercise 21.9

In [10]:
notional = 100
risk_free_rate = .05 # continuous compounding
pr_downgrade = 1. - .004 - .09 - .6574
T = 1

In [11]:
price = notional * exp(-risk_free_rate * T) * pr_downgrade
print('The price of the credit derivative (using historical probabilities) is: {:s}'
     .format(locale.currency(price, grouping=True)))

The price of the credit derivative (using historical probabilities) is: $23.65


### Exercise 21.10

In [14]:
ead = 10_000_000
pr_default = .01
recovery_rate = .4
percentile = .995
ρ = .2

In [15]:
wcdr = norm.cdf((norm.ppf(pr_default) + sqrt(ρ)*norm.ppf(percentile)) / sqrt(1-ρ))
print(f'WCDR: {wcdr:.4f}')

WCDR: 0.0946


In [19]:
var = (wcdr - pr_default) * ead * (1 - recovery_rate)
print('{:.1%} {:d}-year credit VaR: {:s}'.format(percentile, T, locale.currency(var, grouping=True)))

99.5% 1-year credit VaR: $507,527.27
